In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import os
working_dir = os.getcwd()
if working_dir.endswith('\\ADProgModel\\notebooks'):
    os.chdir('../')

#### Model functions

In [ ]:
def get_info_and_cog(reg1_mri, reg2_mri, gamma_, lambda_):

    reg1_info_range = np.arange(0,10.5, step=0.1)
    reg2_info_range = np.arange(0,10.5, step=0.1)

    C_task = 10

    current_reward_max = -np.Inf
    reward_list = []
    
    for info1 in reg1_info_range:
        for info2 in reg2_info_range:
            cog_val = info1 + info2
            Y_val = np.array([gamma_*info1/reg1_mri, gamma_*info2/reg2_mri])
            
            power_factor = np.clip(-C_task + cog_val, a_min=0, a_max=None)
            factor = 100**power_factor
            reward_val = -(np.abs(C_task - cog_val)*factor*lambda_ + Y_val.sum()) 
            
            reward_list.append(reward_val)
            
            if reward_val > current_reward_max:
                best_info_dist = np.array([info1, info2])
                best_cog_val = cog_val
                best_Y_val = Y_val
                current_reward_max = reward_val
                
    return best_info_dist, best_cog_val, best_Y_val

In [ ]:
# deterioration associated with metabolism and pathology

def get_new_structure(D_old, Y_V, X_V_old, phi_old, beta, alpha_1, alpha_2):
    # evolution of amyloid deposition over time
    # dD/dt = -beta*H*D
    D_new = D_old - beta* H @ D_old
    phi_new = phi_old + D_new
    
    # new health of brain regions
    delta_X = -alpha_1*D_new -alpha_2*Y_V
    X_V_new = X_V_old + delta_X
    
    return X_V_new, D_new, phi_new

In [ ]:
# change in cognition and health over time

def simulate_trajectories(dat_X_V, dat_D, beta_, alpha1_, alpha2_, gamma_, lambda_):

    total_time = 10
    num_regions = len(dat_X_V)
    
    infodist_mat = np.zeros((num_regions, total_time+1))
    struct_mat = np.zeros((num_regions, total_time+1))
    func_mat = np.zeros((num_regions, total_time+1))
    amyloid_mat = np.zeros((num_regions, total_time+1))
    cognition_vec = np.zeros(total_time+1)

    dat_phi_old = dat_D.copy()
    dat_X_V_old = dat_X_V.copy()
    dat_D_old = dat_D.copy()
    
    for tm in range(total_time+1):
        
        # cognition value
        # check whether any dat_X_V is zero
        if np.any(dat_X_V_old <= 0):
            break
        else:
            dat_I_V, cog_val, dat_Y_V = get_info_and_cog(dat_X_V_old[0], dat_X_V_old[1], gamma_, lambda_)
        
        # store data
        struct_mat[:,tm] = dat_X_V_old
        amyloid_mat[:,tm] = dat_phi_old
        infodist_mat[:,tm] = dat_I_V
        func_mat[:,tm] = dat_Y_V
        cognition_vec[tm] = cog_val
        
        # update health
        dat_X_V_new, dat_D_new, dat_phi_new = get_new_structure(dat_D_old, dat_Y_V, dat_X_V_old, dat_phi_old, beta_, alpha1_, alpha2_)
        
        # update variables
        dat_X_V_old = dat_X_V_new.copy()
        dat_D_old = dat_D_new.copy()
        dat_phi_old = dat_phi_new.copy()
        
    return struct_mat, func_mat, infodist_mat, amyloid_mat, cognition_vec

In [ ]:
def get_D_init(phi_t, beta_, tpo_):
    lamb, U = np.linalg.eig(H)
    diag_array = np.diag([lamb[0]*np.exp(-lamb[0]*beta_*tpo_)/(1-np.exp(-lamb[0]*beta_*tpo_)), 1/(beta_*tpo_)])
    mult = U.dot(diag_array).dot(U.T)
    mult = U.dot(diag_array).dot(U.T)
    D0_init = beta_*(mult).dot(phi_t.T)   
    
    return D0_init

### ADNI

#### Load the data

In [ ]:
# load the data
splitnum = 0
file_name = 'adni_split%d.xls'%(splitnum)
par_file_name = 'adni_split%d_parameters.xls'%(splitnum)
save_full_file_name = 'output/control_mod_adni_split%d.xlsx'%(splitnum)

testdf = pd.read_excel('dataset/processed/'+file_name, sheet_name='test')
paramdf = pd.read_excel('dataset/processed/'+par_file_name, sheet_name='MMSE_norm_PTGENDER_APOEPOS')

cogcolname = 'MMSE_norm_pred'

In [ ]:
gamma_val = 1
lambda_val = 1
dat_X_E = np.array([[0,1],[1,0]]) # adjacency matrix
H = np.diag(np.sum(dat_X_E, axis=1)) - dat_X_E
mri_colname = ['mri_FRONT_norm', 'mri_HIPPO_norm']
suvr_colname = ['FRONTAL_SUVR', 'HIPPOCAMPAL_SUVR']

resdf = pd.DataFrame({})

for patname in testdf.RID.unique():
    print(patname)
    
    # get parameters
    tpo_val = paramdf.loc[paramdf.RID==patname].tpo_estm.values[0]
    beta_val = paramdf.loc[paramdf.RID==patname].beta_estm.values[0]
    alpha1_val = paramdf.loc[paramdf.RID==patname].alpha1_estm.values[0]
    alpha2_val = paramdf.loc[paramdf.RID==patname].alpha2_gamma_estm.values[0]/gamma_val

    # compute initial X
    subdata_X_V = testdf.loc[(testdf.RID==patname)&(testdf.Years==0), mri_colname].values[0]

    # compute the D value
    subdata_phi = testdf.loc[(testdf.RID==patname)&(testdf.Years==0), suvr_colname].values[0]
    subdata_D = get_D_init(subdata_phi, beta_val, tpo_val)

    # compute trajectory from non-RL model
    X_V_traj, Y_V_traj, I_V_traj, phi_traj, cogn_traj = simulate_trajectories(subdata_X_V, subdata_D, 
                                                                   beta_val, alpha1_val, alpha2_val, gamma_val, lambda_val)

    # store results in a df
    subresdf = pd.DataFrame({'RID':11*[patname], 'Years':np.arange(0,11),
                             'reg1_mri_pred': X_V_traj[0,:], 
                             'reg2_mri_pred': X_V_traj[1,:], 
                             'reg1_fdg_pred': Y_V_traj[0,:], 
                             'reg2_fdg_pred': Y_V_traj[1,:], 
                             'reg1_info_pred': I_V_traj[0,:], 
                             'reg2_info_pred': I_V_traj[1,:], 
                             cogcolname: cogn_traj})

    # combine results with other subjects results
    resdf = pd.concat((resdf, subresdf), axis=0, ignore_index=True)

In [ ]:
fresdf = testdf.merge(resdf, how='right', on=['RID','Years'])

In [ ]:
save_full_file_name

In [ ]:
fresdf.to_excel(save_full_file_name, index=False)

### Synthetic data

In [ ]:
gamma_val = 1
lambda_val = 8

# load the data
splitnum = 0
file_name = 'synthetic_split%d.xls'%(splitnum)
save_full_file_name = 'output/control_mod_synthetic_split%d_lambda%d.xlsx'%(splitnum, lambda_val)

testdf = pd.read_excel('dataset/processed/'+file_name, sheet_name='test')

cogcolname = 'cogsc_pred'

In [ ]:
dat_X_E = np.array([[0,1],[1,0]]) # adjacency matrix
H = np.diag(np.sum(dat_X_E, axis=1)) - dat_X_E
mri_colname = ['reg1_mri', 'reg2_mri']
suvr_colname = ['reg1_av45', 'reg2_av45']

resdf = pd.DataFrame({})

for patname in testdf.RID.unique():
    print(patname)
    
    # get parameters
    tpo_val = testdf.loc[(testdf.RID==patname) & (testdf.Years==0)].tpo.values[0] + 0.5
    beta_val = testdf.loc[(testdf.RID==patname) & (testdf.Years==0)].beta.values[0]
    alpha1_val = testdf.loc[(testdf.RID==patname) & (testdf.Years==0)].alpha1.values[0]
    alpha2_val = testdf.loc[(testdf.RID==patname) & (testdf.Years==0)].alpha2.values[0]

    # compute initial X
    subdata_X_V = testdf.loc[(testdf.RID==patname)&(testdf.Years==0), mri_colname].values[0]

    # compute the D value
    subdata_phi = testdf.loc[(testdf.RID==patname)&(testdf.Years==0), suvr_colname].values[0]
    subdata_D = get_D_init(subdata_phi, beta_val, tpo_val)

    # compute trajectory from non-RL model
    X_V_traj, Y_V_traj, I_V_traj, phi_traj, cogn_traj = simulate_trajectories(subdata_X_V, subdata_D, 
                                                                   beta_val, alpha1_val, alpha2_val, gamma_val, lambda_val)

    # store results in a df
    subresdf = pd.DataFrame({'RID':11*[patname], 'Years':np.arange(0,11),
                             'reg1_mri_pred': X_V_traj[0,:], 
                             'reg2_mri_pred': X_V_traj[1,:], 
                             'reg1_fdg_pred': Y_V_traj[0,:], 
                             'reg2_fdg_pred': Y_V_traj[1,:], 
                             'reg1_info_pred': I_V_traj[0,:], 
                             'reg2_info_pred': I_V_traj[1,:], 
                             cogcolname: cogn_traj})

    # combine results with other subjects results
    resdf = pd.concat((resdf, subresdf), axis=0, ignore_index=True)

In [ ]:
fresdf = testdf.merge(resdf, how='right', on=['RID','Years'])

In [ ]:
fresdf.to_excel(save_full_file_name, index=False)